In [42]:
# Семинар 16: эндогенность и тестирование инструментов
import numpy as np
from linearmodels.datasets import mroz
from statsmodels.api import add_constant

print(mroz.DESCR)
data = mroz.load()
data = add_constant(data, has_constant="add")
print(data)


T.A. Mroz (1987), "The Sensitivity of an Empirical Model of Married Women's
Hours of Work to Economic and Statistical Assumptions," Econometrica 55,
765-799.

nlf        1 if in labor force, 1975
hours      hours worked, 1975
kidslt6    # kids < 6 years
kidsge6    # kids 6-18
age        woman's age in yrs
educ       years of schooling
wage       estimated wage from earns., hours
repwage    reported wage at interview in 1976
hushrs     hours worked by husband, 1975
husage     husband's age
huseduc    husband's years of schooling
huswage    husband's hourly wage, 1975
faminc     family income, 1975
mtr        fed. marginal tax rate facing woman
motheduc   mother's years of schooling
fatheduc   father's years of schooling
unem       unem. rate in county of resid.
city       =1 if live in SMSA
exper      actual labor mkt exper
nwifeinc   (faminc - wage*hours)/1000
lwage      log(wage)
expersq    exper^2

     const  inlf  hours  kidslt6  kidsge6  age  educ    wage  repwage  hushrs  \
0   

/Users/polinapogorelova/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [48]:
# Для начала оценим модель для зарплаты, игнорирую эндогенность переменной educ, с помошью МНК
from linearmodels.iv import IV2SLS

res_ols = IV2SLS(data.lwage, data[["const", "educ"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:                  lwage   R-squared:                      0.1179
Estimator:                        OLS   Adj. R-squared:                 0.1158
No. Observations:                 428   F-statistic:                    57.196
Date:                Mon, Jan 23 2023   P-value (F-stat)                0.0000
Time:                        11:10:11   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const         -0.1852     0.1848    -1.0022     0.3163     -0.5474      0.1770
educ           0.1086     0.0144     7.5628     0.00

In [49]:
# Теперь учтем эндогенность переменной educ, и реализуем для нее метод инструментальных переменных, рассмотрев в качестве 
# инструмента переменную fathereduc (образование отца, в годах)

# Проделаем вручную первый шаг из двухшаговой процедуры МНК, чтобы првоерить инструменты на релевантность
# Шаг 1: оценим регрессию эндогенной переменной educ на инструмент fathereduc
res_first = IV2SLS(data.educ, data[["const", "fatheduc"]], None, None).fit(
    cov_type="unadjusted"
)
print(res_first)

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.1958
Estimator:                        OLS   Adj. R-squared:                 0.1947
No. Observations:                 753   F-statistic:                    183.30
Date:                Mon, Jan 23 2023   P-value (F-stat)                0.0000
Time:                        11:11:09   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          9.7990     0.1983     49.422     0.0000      9.4104      10.188
fatheduc       0.2824     0.0209     13.539     0.00

In [54]:
# Теперь автоматически оценим параметры с помощью МИП

res_iv = IV2SLS(data.lwage, data.const, data.educ, data.fatheduc).fit(
    cov_type = "unadjusted"
)
print(res_second)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                  lwage   R-squared:                      0.0934
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0913
No. Observations:                 428   F-statistic:                    2.8487
Date:                Mon, Jan 23 2023   P-value (F-stat)                0.0915
Time:                        11:12:17   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.4411     0.4451     0.9911     0.3216     -0.4312      1.3134
educ           0.0592     0.0351     1.6878     0.09

In [55]:
# Проверим инструменты на валидность с помощью теста Ву-Хаусмана

print(res_second.wu_hausman())

# Основная гипотеза об экзогенности инструментов не отвергается, значит, мы выбрали подходящие инструменты.

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 2.4703
P-value: 0.1168
Distributed: F(1,425)


In [37]:
# Теперь попробуем использовать два инструмента motheduc и fatheduc для эндогенной переменной educ

endog = ["educ"]
exog = ["const"]
instr = ["motheduc",
         "fatheduc"]

In [56]:
# Реализуем двухшаговый МНК

# Шаг 1: оценим регрессию эндогенной переменной educ на инструменты motheduc и fathereduc
res_first_2ols = IV2SLS(data.educ, data[instr+exog], None, None).fit(
    cov_type="unadjusted"
)
print(res_first_2ols)

                            OLS Estimation Summary                            
Dep. Variable:                   educ   R-squared:                      0.2450
Estimator:                        OLS   Adj. R-squared:                 0.2430
No. Observations:                 753   F-statistic:                    244.31
Date:                Mon, Jan 23 2023   P-value (F-stat)                0.0000
Time:                        11:20:01   Distribution:                  chi2(2)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
motheduc       0.1833     0.0262     7.0049     0.0000      0.1320      0.2346
fatheduc       0.1834     0.0247     7.4366     0.00

In [57]:
# Получим 2OLS оценку параметров

res_2ols = IV2SLS(np.log(data.wage), data[exog], data[endog], data[instr]).fit(
    cov_type="unadjusted"
)
print(res_2ols)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                   wage   R-squared:                      0.0841
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0820
No. Observations:                 428   F-statistic:                    2.4752
Date:                Mon, Jan 23 2023   P-value (F-stat)                0.1157
Time:                        11:20:24   Distribution:                  chi2(1)
Cov. Estimator:            unadjusted                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.5510     0.4076     1.3518     0.1764     -0.2479      1.3500
educ           0.0505     0.0321     1.5733     0.11

In [61]:
# Проверим:
# 1) является ли эндогенной переменная educ с помощью процедуры Ву-Хаусмана (H0: переменная educ экзогенна, используем МНК-оценки)
print(res_2ols.wu_hausman())

# Основная гипотеза об экзогенности инструментов в обоих тестах отвергается на 1% уровне значимости, значит, мы выбрали подходящие инструменты.

Wu-Hausman test of exogeneity
H0: All endogenous variables are exogenous
Statistic: 4.3296
P-value: 0.0381
Distributed: F(1,425)


In [62]:
# Для сравнения поместим результаты оценивания с помощью МНК и 2МНК в одун таблицу
from linearmodels.iv import compare

print(compare({"OLS": res_ols, "IV": res_iv, "2OLS": res_2ols}))

                         Model Comparison                        
                                OLS             IV           2OLS
-----------------------------------------------------------------
Dep. Variable                 lwage          lwage           wage
Estimator                       OLS        IV-2SLS        IV-2SLS
No. Observations                428            428            428
Cov. Est.                unadjusted     unadjusted     unadjusted
R-squared                    0.1179         0.0934         0.0841
Adj. R-squared               0.1158         0.0913         0.0820
F-statistic                  57.196         2.8487         2.4752
P-value (F-stat)          3.941e-14         0.0915         0.1157
==================     ============   ============   ============
const                       -0.1852         0.4411         0.5510
                          (-1.0022)       (0.9911)       (1.3518)
educ                         0.1086         0.0592         0.0505
          